In [1]:
#### importing necessary libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

df = pd.read_csv('Effort estimation data set.csv')
df

#### Splitting the data into training and testing sets

X = df.drop(columns=['Effort (Actual)'])
y = df['Effort (Actual)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Linear Regression

lr = LinearRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

#### Decision Tree Regressor

dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)

#### MLP Regressor

mlp = MLPRegressor()
mlp.fit(X_train, y_train)
mlp_pred = mlp.predict(X_test)

#### SVR

svr = SVR()
svr.fit(X_train, y_train)
svr_pred = svr.predict(X_test)

#### SMO with sigmoid kernel (SVR with sigmoid kernel)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svr_sigmoid = SVR(kernel='sigmoid')
svr_sigmoid.fit(X_train_scaled, y_train)
svr_sigmoid_pred = svr_sigmoid.predict(X_test_scaled)

#### SMO with polynomial kernel (SVR with polynomial kernel)

svr_poly = SVR(kernel='poly')
svr_poly.fit(X_train_scaled, y_train)
svr_poly_pred = svr_poly.predict(X_test_scaled)

#### SMO with RBF kernel (SVR with RBF kernel)

svr_rbf = SVR(kernel='rbf')
svr_rbf.fit(X_train_scaled, y_train)
svr_rbf_pred = svr_rbf.predict(X_test_scaled)

#### Define a function to calculate performance metrics

# Define a function to calculate normalized mean squared error
def normalized_mean_squared_error(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) / np.var(y_true)

def evaluate_performance(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    nmse = normalized_mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mse, mape, nmse, r2

# Evaluate performance metrics for the models
models = [lr, dt, mlp, svr, svr_sigmoid, svr_poly, svr_rbf]
predictions = [lr_pred, dt_pred, mlp_pred, svr_pred, svr_sigmoid_pred, svr_poly_pred, svr_rbf_pred]
model_names = ['Linear Regression', 'Decision Tree Regressor', 'MLP Regressor', 'SVR', 'SMO with Sigmoid Kernel', 'SMO with polynomial Kernel', 'SMO with RBF Kernel']

results = []
for model, pred, name in zip(models, predictions, model_names):
    mae, mse, mape, nmse, r2 = evaluate_performance(y_test, pred)
    results.append([name, mae, mse, mape, nmse, r2])

# Create DataFrame from results
df_results = pd.DataFrame(results, columns=["Model", "Mean Absolute Error", "Mean Squared Error", "Mean Absolute Percentage Error", "Normalized Mean Squared Error", "R^2 Score"])
df_results

from sklearn.model_selection import GridSearchCV, cross_val_score

# Define the parameter grid for each model
param_grid_dt = {
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_grid_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (200,)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'lbfgs'],
    'alpha': [0.0001, 0.001, 0.01],
    'batch_size': [32, 64, 128],
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [200, 300, 400]
}

param_grid_svr = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4],
    'epsilon': [0.1, 0.2, 0.3]
}

# Perform Grid Search for Decision Tree Regressor
grid_dt = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=param_grid_dt, scoring='neg_mean_squared_error', cv=5)
grid_dt.fit(X_train, y_train)
dt_tuned = grid_dt.best_estimator_
dt_tuned.fit(X_train, y_train)
dt_pred_tuned = dt_tuned.predict(X_test)

# Perform Grid Search for MLP Regressor
grid_mlp = GridSearchCV(estimator=MLPRegressor(), param_grid=param_grid_mlp, scoring='neg_mean_squared_error', cv=5)
grid_mlp.fit(X_train, y_train)
mlp_tuned = grid_mlp.best_estimator_
mlp_tuned.fit(X_train, y_train)
mlp_pred_tuned = mlp_tuned.predict(X_test)

# Perform Grid Search for SVR
grid_svr = GridSearchCV(estimator=SVR(), param_grid=param_grid_svr, scoring='neg_mean_squared_error', cv=5)
grid_svr.fit(X_train_scaled, y_train)
svr_tuned = grid_svr.best_estimator_
svr_tuned.fit(X_train_scaled, y_train)
svr_pred_tuned = svr_tuned.predict(X_test_scaled)

# Perform Grid Search for SMO with sigmoid kernel (SVR with sigmoid kernel)
param_grid_svr_sigmoid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto'],
    'epsilon': [0.1, 0.2, 0.3]
}
grid_svr_sigmoid = GridSearchCV(estimator=SVR(kernel='sigmoid'), param_grid=param_grid_svr_sigmoid, scoring='neg_mean_squared_error', cv=5)
grid_svr_sigmoid.fit(X_train_scaled, y_train)
svr_sigmoid_tuned = grid_svr_sigmoid.best_estimator_
svr_sigmoid_tuned.fit(X_train_scaled, y_train)
svr_sigmoid_pred_tuned = svr_sigmoid_tuned.predict(X_test_scaled)

# Perform Grid Search for SMO with polynomial kernel (SVR with polynomial kernel)
param_grid_svr_poly = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4],
    'epsilon': [0.1, 0.2, 0.3]
}
grid_svr_poly = GridSearchCV(estimator=SVR(kernel='poly'), param_grid=param_grid_svr_poly, scoring='neg_mean_squared_error', cv=5)
grid_svr_poly.fit(X_train_scaled, y_train)
svr_poly_tuned = grid_svr_poly.best_estimator_
svr_poly_tuned.fit(X_train_scaled, y_train)
svr_poly_pred_tuned = svr_poly_tuned.predict(X_test_scaled)

# Perform Grid Search for SMO with RBF kernel (SVR with RBF kernel)
param_grid_svr_rbf = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto'],
    'epsilon': [0.1, 0.2, 0.3]
}
grid_svr_rbf = GridSearchCV(estimator=SVR(kernel='rbf'), param_grid=param_grid_svr_rbf, scoring='neg_mean_squared_error', cv=5)
grid_svr_rbf.fit(X_train_scaled, y_train)
svr_rbf_tuned = grid_svr_rbf.best_estimator_
svr_rbf_tuned.fit(X_train_scaled, y_train)
svr_rbf_pred_tuned


C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer

C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/sta

C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the

C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERAT

C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _ch

...

C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERAT

C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERAT

C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERAT

C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:603: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:603: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfg

KeyboardInterrupt: 

<b>Decision Tree Regressor</b> has the lowest MAE, MSE, MAPE, and NMSE among the models, indicating better accuracy and performance compared to other models.

Linear Regression and SMO with polynomial Kernel also show relatively good performance with lower MAE, MSE, and R^2 Score compared to other models.

SVR and SMO with RBF Kernel have the highest MAE, MSE, MAPE, and NMSE, indicating poorer performance compared to other models.

SVR and SMO with Sigmoid Kernel also have high MAE, MSE, and MAPE, but slightly better R^2 Score compared to SVR and SMO with RBF Kernel.

Overall, Decision Tree Regressor appears to be the best-performing model based on the provided metrics, followed by Linear Regression and SMO with polynomial Kernel. SVR and SMO with RBF Kernel show the poorest performance among the models.